## Introduction to Iceberg Architecture

In [1]:
!pip3 install -r requirements.txt

  Cloning https://github.com/fastforwardlabs/cmlbootstrap to /tmp/pip-install-sd37j61q/cmlbootstrap_f87bc57457124a2bacc9a6c53274f060
  Running command git clone -q https://github.com/fastforwardlabs/cmlbootstrap /tmp/pip-install-sd37j61q/cmlbootstrap_f87bc57457124a2bacc9a6c53274f060


#### Launching a Spark Session with Iceberg

In [2]:
import cml.data_v1 as cmldata

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

23/09/04 03:00:41 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/04 03:00:41 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/04 03:00:42 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/04 

+--------------------+
|           namespace|
+--------------------+
|         01_car_data|
|           01_car_dw|
|              adb101|
|            airlines|
|        airlines_csv|
|    airlines_iceberg|
|airlines_iceberg_...|
|      airlines_mjain|
|          airquality|
|                ajvp|
|          atlas_demo|
|            bankdemo|
|          bca_jps_l0|
|        cde_workshop|
|             cdedemo|
|        cdp_overview|
|      ceht_open_data|
|        ceht_scratch|
| ceht_transportation|
|        cgsifacebook|
+--------------------+
only showing top 20 rows



In [3]:
spark.sparkContext.getConf().getAll()

23/09/04 03:00:56 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/04 03:00:56 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


[('spark.kubernetes.driver.pod.name', '5esiffpo4nllmdwy'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.network.crypto.enabled', 'true'),
 ('spark.sql.hive.hwc.execution.mode', 'spark'),
 ('spark.jars',
  '/opt/spark/optional-lib/hive-warehouse-connector-assembly.jar,/opt/spark/optional-lib/iceberg-hive-runtime.jar,/opt/spark/optional-lib/iceberg-spark-runtime.jar'),
 ('spark.kerberos.renewal.credentials', 'ccache'),
 ('spark.sql.catalog.spark_catalog',
  'org.apache.iceberg.spark.SparkSessionCatalog'),
 ('spark.dynamicAllocation.maxExecutors', '49'),
 ('spark.eventLog.dir', 'file:///sparkeventlogs'),
 ('spark.hadoop.yarn.resourcemanager.principal', 'pauldefusco'),
 ('spark.app.startTime', '1693796442306'),
 ('spark.ui.port', '20049'),
 ('spark.app.id', 'spark-application-1693796444041'),
 ('spark.driver.bindAddress', '100.100.216.56'),
 ('spark.kubernetes.driver.annotation.cluster-autoscaler.kubernetes.io/safe-to-evict',
  'false'),
 ('spark.yarn.access.hadoopFileSystems',
  's3a://g

### Iceberg Architecture

![alt text](../img/iceberg-metadata.png)

#### Iceberg Catalog

Iceberg comes with catalogs that enable SQL commands to manage tables and load them by name. Catalogs are configured using properties under spark.sql.catalog.(catalog_name).

In [4]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [5]:
# Create a new database
#spark.sql("DROP DATABASE IF EXISTS spark_catalog.lakehouse")
spark.sql("CREATE DATABASE IF NOT EXISTS spark_catalog.lakehouse")
spark.sql("USE spark_catalog.lakehouse")

DataFrame[]

In [6]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|lakehouse|
+-------------+---------+



#### Create an Iceberg Table with Spark SQL

In [7]:
spark.sql("DROP TABLE IF EXISTS lakehouse.coffees_table_2 PURGE")

23/09/04 03:00:56 WARN HiveMetaStoreClient: Failed to connect to the MetaStore Server...
23/09/04 03:01:00 WARN ExecutorPodsSnapshotsStoreImpl: Exception when notifying snapshot subscriber.
io.fabric8.kubernetes.client.KubernetesClientException: Failure executing: POST at: https://172.20.0.1/api/v1/namespaces/mlx-user-16/pods. Message: pods "cdsw-5esiffpo4nllmdwy-exec-1" already exists. Received status: Status(apiVersion=v1, code=409, details=StatusDetails(causes=[], group=null, kind=pods, name=cdsw-5esiffpo4nllmdwy-exec-1, retryAfterSeconds=null, uid=null, additionalProperties={}), kind=Status, message=pods "cdsw-5esiffpo4nllmdwy-exec-1" already exists, metadata=ListMeta(_continue=null, remainingItemCount=null, resourceVersion=null, selfLink=null, additionalProperties={}), reason=AlreadyExists, status=Failure, additionalProperties={}).
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.requestFailure(OperationSupport.java:681)
	at io.fabric8.kubernetes.client.dsl.base.Operatio

DataFrame[]

# TEST 2 - MOR

In [8]:
spark.sql("CREATE TABLE IF NOT EXISTS coffees_table_2 (coffee_id BIGINT, coffee_size STRING, coffee_sale_ts TIMESTAMP)\
          USING ICEBERG\
          PARTITIONED BY (months(coffee_sale_ts))\
          TBLPROPERTIES ('write.delete.mode'='merge-on-read',\
                          'write.update.mode'='merge-on-read',\
                          'write.merge.mode'='merge-on-read',\
                          'format-version' = '2')")

DataFrame[]

#### Verify that a Metadata JSON file has been created under the Metadata directory

In [9]:
metadata_path = "warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2"

In [10]:
import boto3

s3 = boto3.resource('s3')

my_bucket = s3.Bucket("go01-demo")

for object_summary in my_bucket.objects.filter(Prefix=metadata_path):
    #print(object_summary.key)
    metadata_file = object_summary.key
    
print("Metadata File Path: {}".format(metadata_file))

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/00000-ae122cd1-2725-4c19-b834-7461c9121690.metadata.json


In [11]:
import pandas as pd
spark.read.option("multiline","true").json("s3a://go01-demo/" + "warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata").toPandas()

,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-sequence-number,last-updated-ms,location,metadata-log,partition-specs,properties,schemas,snapshot-log,snapshots,sort-orders,statistics,table-uuid
0,0,-1,0,0,2,3,1000,0,1693796481590,s3a://go01-demo/warehouse/tablespace/external/...,[],"[([Row(field-id=1000, name='coffee_sale_ts_mon...","(pauldefusco, merge-on-read, merge-on-read, me...","[([Row(id=1, name='coffee_id', required=False,...",[],[],"[([], 0)]",[],6cc4ed40-098f-4f0b-b5d8-e92fbbe8cc4b


![alt text](../img/s3_metadata.png)

#### Notice that no snapshots or other files have been created as data has not yet been inserted.

In [12]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.history").show()

+---------------+-----------+---------+-------------------+
|made_current_at|snapshot_id|parent_id|is_current_ancestor|
+---------------+-----------+---------+-------------------+
+---------------+-----------+---------+-------------------+



In [13]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.snapshots;").show()

+------------+-----------+---------+---------+-------------+-------+
|committed_at|snapshot_id|parent_id|operation|manifest_list|summary|
+------------+-----------+---------+---------+-------------+-------+
+------------+-----------+---------+---------+-------------+-------+



In [14]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.files;").show()

+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
|content|file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|column_sizes|value_counts|null_value_counts|nan_value_counts|lower_bounds|upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+



In [15]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.manifests;").show()

+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
|content|path|length|partition_spec_id|added_snapshot_id|added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries|
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+



In [16]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.all_data_files;").show()

+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
|content|file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|column_sizes|value_counts|null_value_counts|nan_value_counts|lower_bounds|upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+



In [17]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.all_manifests;").show()

+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+---------------------+
|content|path|length|partition_spec_id|added_snapshot_id|added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries|reference_snapshot_id|
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+---------------------+
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+-------

### Table Insert

In [18]:
from pyspark.sql.functions import date_format

In [19]:
#Coffee_id = 1, Coffee_size = venti, coffee_sale_ts = 2023-09-01

spark.sql("INSERT INTO lakehouse.coffees_table_2 VALUES (1, 'venti', cast(date_format('2023-09-01 10:00:00', 'yyyy-MM-dd HH:mm:ss') as timestamp))")

DataFrame[]

#### Data has been added to the data folder

In [20]:
QUERY = "select h.made_current_at,\
            s.operation,\
            h.snapshot_id,\
            h.is_current_ancestor,\
            s.summary['spark.app.id']\
        from lakehouse.coffees_table.history h\
        join lakehouse.coffees_table.snapshots s\
            on h.snapshot_id = s.snapshot_id\
            order by made_current_at;"

In [21]:
spark.sql(QUERY).toPandas()

23/09/04 03:01:31 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


,made_current_at,operation,snapshot_id,is_current_ancestor,summary[spark.app.id]
0,2023-09-04 02:47:21.766,append,6464031631726591033,True,spark-application-1693795583203
1,2023-09-04 02:47:43.813,overwrite,5876718107486774833,True,spark-application-1693795583203


#### Notice there are now two json files and two avro files. 

The first json file is the metadata file created when the table was created. This is the metata file prefixed by 00000. The second json file is the new metadata file reflecting the insert of one row. This is the metadata file prefixed by 00001.

The avro file with the "snap" prefix is the manifest list. The other avro file created is the corresponding manifest file.

In [22]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

print("Current Metadata Files: \n")
for object_summary in my_bucket.objects.filter(Prefix=metadata_path+"/metadata"):
    print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)

Current Metadata Files: 

warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/00000-ae122cd1-2725-4c19-b834-7461c9121690.metadata.json

warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/00001-f20716ed-0e61-42e5-8db0-51f1d984a719.metadata.json

warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/52754ebc-cb8c-41ba-8fd0-0330469eb626-m0.avro

warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/snap-476467335916624763-1-52754ebc-cb8c-41ba-8fd0-0330469eb626.avro



Showing Metadata Files (JSON)

In [23]:
import pandas as pd

print("Showing " + metadata_file_list[0])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[0]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/00000-ae122cd1-2725-4c19-b834-7461c9121690.metadata.json


,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-sequence-number,last-updated-ms,location,metadata-log,partition-specs,properties,schemas,snapshot-log,snapshots,sort-orders,statistics,table-uuid
0,0,-1,0,0,2,3,1000,0,1693796481590,s3a://go01-demo/warehouse/tablespace/external/...,[],"[([Row(field-id=1000, name='coffee_sale_ts_mon...","(pauldefusco, merge-on-read, merge-on-read, me...","[([Row(id=1, name='coffee_id', required=False,...",[],[],"[([], 0)]",[],6cc4ed40-098f-4f0b-b5d8-e92fbbe8cc4b


In [24]:
print("Showing " + metadata_file_list[1])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[1]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/00001-f20716ed-0e61-42e5-8db0-51f1d984a719.metadata.json


,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-sequence-number,last-updated-ms,location,metadata-log,partition-specs,properties,refs,schemas,snapshot-log,snapshots,sort-orders,statistics,table-uuid
0,0,476467335916624763,0,0,2,3,1000,1,1693796490216,s3a://go01-demo/warehouse/tablespace/external/...,[(s3a://go01-demo/warehouse/tablespace/externa...,"[([Row(field-id=1000, name='coffee_sale_ts_mon...","(pauldefusco, merge-on-read, merge-on-read, me...","((476467335916624763, branch),)","[([Row(id=1, name='coffee_id', required=False,...","[(476467335916624763, 1693796490216)]",[(s3a://go01-demo/warehouse/tablespace/externa...,"[([], 0)]",[],6cc4ed40-098f-4f0b-b5d8-e92fbbe8cc4b


Showing Manifest List (AVRO - prefixed by "SNAP")

In [25]:
print("Showing " + metadata_file_list[3])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[3]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/snap-476467335916624763-1-52754ebc-cb8c-41ba-8fd0-0330469eb626.avro


,manifest_path,manifest_length,partition_spec_id,content,sequence_number,min_sequence_number,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_rows_count,existing_rows_count,deleted_rows_count,partitions
0,s3a://go01-demo/warehouse/tablespace/external/...,7069,0,0,1,1,476467335916624763,1,0,0,1,0,0,"[(False, False, [132, 2, 0, 0], [132, 2, 0, 0])]"


Showing Manifest Files (Avro) i.e. which shows data file locations according to each snapshot_id

In [26]:
print("Showing " + metadata_file_list[2])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[2]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/52754ebc-cb8c-41ba-8fd0-0330469eb626-m0.avro


,status,snapshot_id,sequence_number,file_sequence_number,data_file
0,1,476467335916624763,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."


### Table Merge Into

Create a staging table

In [27]:
spark.sql("DROP TABLE IF EXISTS lakehouse.coffee_staging_2 PURGE")

23/09/04 03:01:46 WARN ExecutorPodsSnapshotsStoreImpl: Exception when notifying snapshot subscriber.
io.fabric8.kubernetes.client.KubernetesClientException: Failure executing: POST at: https://172.20.0.1/api/v1/namespaces/mlx-user-16/pods. Message: pods "cdsw-5esiffpo4nllmdwy-exec-11" already exists. Received status: Status(apiVersion=v1, code=409, details=StatusDetails(causes=[], group=null, kind=pods, name=cdsw-5esiffpo4nllmdwy-exec-11, retryAfterSeconds=null, uid=null, additionalProperties={}), kind=Status, message=pods "cdsw-5esiffpo4nllmdwy-exec-11" already exists, metadata=ListMeta(_continue=null, remainingItemCount=null, resourceVersion=null, selfLink=null, additionalProperties={}), reason=AlreadyExists, status=Failure, additionalProperties={}).
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.requestFailure(OperationSupport.java:681)
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.assertResponseCode(OperationSupport.java:620)
	at io.fabric8.kubernetes.clien

DataFrame[]

In [28]:
spark.sql("CREATE TABLE IF NOT EXISTS lakehouse.coffee_staging_2\
            (coffee_id BIGINT, coffee_size STRING, coffee_sale_ts TIMESTAMP)\
            USING iceberg\
            PARTITIONED BY (months(coffee_sale_ts))")

DataFrame[]

In [29]:
spark.sql("INSERT INTO lakehouse.coffee_staging_2\
            VALUES (1, 'venti', cast(date_format('2023-07-01 11:00:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (2, 'grande', cast(date_format('2023-07-01 11:10:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (3, 'tall', cast(date_format('2023-07-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp))")

#Row: Coffee_id = 1, coffee_size = venti, coffee_sale_ts = 2023-07-01
#Row: Coffee_id = 2, coffee_size = grande, coffee_sale_ts = 2023-07-01
#Row: Coffee_id = 3, coffee_size = tall, coffee_sale_ts = 2023-04-01

DataFrame[]

Merge Into Customers Table

In [30]:
spark.sql("MERGE INTO lakehouse.coffees_table_2 c\
            USING (SELECT * FROM lakehouse.coffee_staging_2) s\
            ON c.coffee_id = s.coffee_id \
            WHEN MATCHED THEN UPDATE SET c.coffee_size = s.coffee_size \
            WHEN NOT MATCHED THEN INSERT *")

DataFrame[]

In [31]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.snapshots;").toPandas()

,committed_at,snapshot_id,parent_id,operation,manifest_list,summary
0,2023-09-04 03:01:30.216,476467335916624763,NaN,append,s3a://go01-demo/warehouse/tablespace/external/...,{'spark.app.id': 'spark-application-1693796444...
1,2023-09-04 03:02:14.727,1376882192344568144,4.764673e+17,overwrite,s3a://go01-demo/warehouse/tablespace/external/...,"{'added-data-files': '2', 'added-position-dele..."


In [32]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.manifests;").toPandas()

,content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,0,s3a://go01-demo/warehouse/tablespace/external/...,7174,0,1376882192344568144,2,0,0,0,0,0,"[(False, False, 2023-07, 2023-09)]"
1,0,s3a://go01-demo/warehouse/tablespace/external/...,7069,0,476467335916624763,1,0,0,0,0,0,"[(False, False, 2023-09, 2023-09)]"
2,1,s3a://go01-demo/warehouse/tablespace/external/...,7098,0,1376882192344568144,0,0,0,1,0,0,"[(False, False, 2023-09, 2023-09)]"


In [33]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.all_data_files;").toPandas()

,content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id
0,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(644,)",1,992,"{1: 39, 2: 40, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...","{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...",None,[4],None,0
1,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(642,)",2,1002,"{1: 46, 2: 49, 3: 47}","{1: 2, 2: 2, 3: 2}","{1: 0, 2: 0, 3: 0}",{},"{1: [2, 0, 0, 0, 0, 0, 0, 0], 2: [103, 114, 97...","{1: [3, 0, 0, 0, 0, 0, 0, 0], 2: [116, 97, 108...",None,[4],None,0
2,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(644,)",1,976,"{1: 33, 2: 34, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...","{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...",None,[4],None,0


#### There is a new metadata file (json) prefixed by 0002.

#### There is a new manifest list file (avro) prefixed by "snap"

#### There is a new manifest file (avro)

In [34]:
import boto3

s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

print("Current Metadata Files: \n")

for object_summary in my_bucket.objects.filter(Prefix=metadata_path+"/metadata"):
    #print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)
    
print(*metadata_file_list, sep = "\n")

print("There is a total of " + str(len(metadata_file_list)) + " files in the Metadata layer")

Current Metadata Files: 

warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/00000-ae122cd1-2725-4c19-b834-7461c9121690.metadata.json
warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/00001-f20716ed-0e61-42e5-8db0-51f1d984a719.metadata.json
warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/00002-aa09db85-7ce4-49cd-9cfe-f0f0547bedbf.metadata.json
warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/1f6b71e2-d5a8-45ad-934b-f98506c29875-m0.avro
warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/1f6b71e2-d5a8-45ad-934b-f98506c29875-m1.avro
warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/52754ebc-cb8c-41ba-8fd0-0330469eb626-m0.avro
warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/snap-1376882192344568144-1-1f6b71e2-d5a8-45ad-934b-f98506c29875.avro
warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/snap-4764673359

Showing Latest (Current) Metadata File (JSON)

In [35]:
print("Showing " + metadata_file_list[2])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[0]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/00002-aa09db85-7ce4-49cd-9cfe-f0f0547bedbf.metadata.json


,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-sequence-number,last-updated-ms,location,metadata-log,partition-specs,properties,schemas,snapshot-log,snapshots,sort-orders,statistics,table-uuid
0,0,-1,0,0,2,3,1000,0,1693796481590,s3a://go01-demo/warehouse/tablespace/external/...,[],"[([Row(field-id=1000, name='coffee_sale_ts_mon...","(pauldefusco, merge-on-read, merge-on-read, me...","[([Row(id=1, name='coffee_id', required=False,...",[],[],"[([], 0)]",[],6cc4ed40-098f-4f0b-b5d8-e92fbbe8cc4b


Showing Latest Manifest List (AVRO - prefixed by "SNAP")

In [36]:
print("Showing " + metadata_file_list[6])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[6]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/snap-1376882192344568144-1-1f6b71e2-d5a8-45ad-934b-f98506c29875.avro


,manifest_path,manifest_length,partition_spec_id,content,sequence_number,min_sequence_number,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_rows_count,existing_rows_count,deleted_rows_count,partitions
0,s3a://go01-demo/warehouse/tablespace/external/...,7174,0,0,2,2,1376882192344568144,2,0,0,3,0,0,"[(False, False, [130, 2, 0, 0], [132, 2, 0, 0])]"
1,s3a://go01-demo/warehouse/tablespace/external/...,7069,0,0,1,1,476467335916624763,1,0,0,1,0,0,"[(False, False, [132, 2, 0, 0], [132, 2, 0, 0])]"
2,s3a://go01-demo/warehouse/tablespace/external/...,7098,0,1,2,2,1376882192344568144,1,0,0,1,0,0,"[(False, False, [132, 2, 0, 0], [132, 2, 0, 0])]"


In [37]:
print("Showing " + metadata_file_list[7])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[7]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/snap-476467335916624763-1-52754ebc-cb8c-41ba-8fd0-0330469eb626.avro


,manifest_path,manifest_length,partition_spec_id,content,sequence_number,min_sequence_number,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_rows_count,existing_rows_count,deleted_rows_count,partitions
0,s3a://go01-demo/warehouse/tablespace/external/...,7069,0,0,1,1,476467335916624763,1,0,0,1,0,0,"[(False, False, [132, 2, 0, 0], [132, 2, 0, 0])]"


In [38]:
print("Showing " + metadata_file_list[6])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[6]).toPandas()['manifest_path'][0]

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/snap-1376882192344568144-1-1f6b71e2-d5a8-45ad-934b-f98506c29875.avro


's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/1f6b71e2-d5a8-45ad-934b-f98506c29875-m0.avro'

In [39]:
print("Showing " + metadata_file_list[6])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[6]).toPandas()['manifest_path'][1]

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/snap-1376882192344568144-1-1f6b71e2-d5a8-45ad-934b-f98506c29875.avro


's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/52754ebc-cb8c-41ba-8fd0-0330469eb626-m0.avro'

Showing Manifest Files (Avro) i.e. list of table partitions mapped to snapshot ID

In [40]:
print("Showing " + metadata_file_list[3])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[3]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/1f6b71e2-d5a8-45ad-934b-f98506c29875-m0.avro


,status,snapshot_id,sequence_number,file_sequence_number,data_file
0,1,1376882192344568144,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."
1,1,1376882192344568144,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."


In [41]:
print("Showing " + metadata_file_list[4])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/1f6b71e2-d5a8-45ad-934b-f98506c29875-m1.avro


,status,snapshot_id,sequence_number,file_sequence_number,data_file
0,1,1376882192344568144,NaN,NaN,"(1, s3a://go01-demo/warehouse/tablespace/exter..."


In [42]:
print("Showing " + metadata_file_list[5])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[5]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/52754ebc-cb8c-41ba-8fd0-0330469eb626-m0.avro


,status,snapshot_id,sequence_number,file_sequence_number,data_file
0,1,476467335916624763,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."


In [45]:
print("Showing " + metadata_file_list[3])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[3]).toPandas()['data_file'][0]

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/1f6b71e2-d5a8-45ad-934b-f98506c29875-m0.avro


Row(content=0, file_path='s3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/data/coffee_sale_ts_month=2023-09/00008-831-1665f868-1b85-4142-b416-ba62d6e7be4c-00001.parquet', file_format='PARQUET', partition=Row(coffee_sale_ts_month=644), record_count=1, file_size_in_bytes=992, column_sizes=[Row(key=1, value=39), Row(key=2, value=40), Row(key=3, value=39)], value_counts=[Row(key=1, value=1), Row(key=2, value=1), Row(key=3, value=1)], null_value_counts=[Row(key=1, value=0), Row(key=2, value=0), Row(key=3, value=0)], nan_value_counts=[], lower_bounds=[Row(key=1, value=bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00')), Row(key=2, value=bytearray(b'venti')), Row(key=3, value=bytearray(b'\x00\xc8A9I\x04\x06\x00'))], upper_bounds=[Row(key=1, value=bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00')), Row(key=2, value=bytearray(b'venti')), Row(key=3, value=bytearray(b'\x00\xc8A9I\x04\x06\x00'))], key_metadata=None, split_offsets=[4], equality_ids=None, sort_order_id=0)

In [46]:
data_file_path = spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[3]).toPandas()['data_file'][0][1]
spark.read.parquet(data_file_path).show()

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        1|      venti|2023-09-01 10:00:00|
+---------+-----------+-------------------+



In [48]:
print("Showing " + metadata_file_list[4])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()['data_file'][0]

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/metadata/1f6b71e2-d5a8-45ad-934b-f98506c29875-m1.avro


Row(content=1, file_path='s3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/data/coffee_sale_ts_month=2023-09/00008-831-48079c9c-66f2-47c9-82a4-a757cabfeb0b-00001.parquet', file_format='PARQUET', partition=Row(coffee_sale_ts_month=644), record_count=1, file_size_in_bytes=1794, column_sizes=[Row(key=2147483546, value=199), Row(key=2147483545, value=33)], value_counts=[Row(key=2147483546, value=1), Row(key=2147483545, value=1)], null_value_counts=[Row(key=2147483546, value=0), Row(key=2147483545, value=0)], nan_value_counts=[], lower_bounds=[Row(key=2147483546, value=bytearray(b's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/data/coffee_sale_ts_month=2023-09/00072-212-473f28c6-00c3-478e-8685-eedf508153f6-00001.parquet')), Row(key=2147483545, value=bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'))], upper_bounds=[Row(key=2147483546, value=bytearray(b's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2

In [64]:
data_file_path = spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()['data_file'][0][1]
spark.read.parquet(data_file_path).show()

+--------------------+---+
|           file_path|pos|
+--------------------+---+
|s3a://go01-demo/w...|  0|
+--------------------+---+



In [67]:
spark.read.parquet(data_file_path).toPandas()['file_path'][0]

's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/data/coffee_sale_ts_month=2023-09/00072-212-473f28c6-00c3-478e-8685-eedf508153f6-00001.parquet'

23/09/04 03:36:41 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed (this is expected if the application is shutting down.)
io.fabric8.kubernetes.client.WatcherException: too old resource version: 13819492 (13823117)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onStatus(AbstractWatchManager.java:265)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onMessage(AbstractWatchManager.java:249)
	at io.fabric8.kubernetes.client.dsl.internal.WatcherWebSocketListener.onMessage(WatcherWebSocketListener.java:93)
	at okhttp3.internal.ws.RealWebSocket.onReadMessage(RealWebSocket.java:323)
	at okhttp3.internal.ws.WebSocketReader.readMessageFrame(WebSocketReader.java:219)
	at okhttp3.internal.ws.WebSocketReader.processNextFrame(WebSocketReader.java:105)
	at okhttp3.internal.ws.RealWebSocket.loopReader(RealWebSocket.java:274)
	at okhttp3.internal.ws.RealWebSocket$2.onResponse(RealWebSocket.java:214)
	at okhttp3.RealCall$AsyncCall.execute(Re

In [62]:
sub_data_file_path = 's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table_2/data/coffee_sale_ts_month=2023-09/00072-212-473f28c6-00c3-478e-8685-eedf508153f6-00001.parquet'

In [63]:
spark.read.parquet(sub_data_file_path).show()

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        1|      venti|2023-09-01 10:00:00|
+---------+-----------+-------------------+



### Time Travel 

In [73]:
snapshots_df = spark.sql("SELECT * FROM lakehouse.customer_table.snapshots;")

In [74]:
first_snapshot = snapshots_df.select("snapshot_id").head(1)[0][0]

#### Validate that the output dataframe only includes one row per the original insert

In [76]:
spark.read\
    .option("snapshot-id", first_snapshot)\
    .format("iceberg")\
    .load("lakehouse.customer_table").toPandas()

,id,state,country,dob
0,1,CA,USA,2000-01-01


In [78]:
avro_tempdf = spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[6]).toPandas()

In [80]:
avro_tempdf.columns

Index(['manifest_path', 'manifest_length', 'partition_spec_id',
       'added_snapshot_id', 'added_data_files_count',
       'existing_data_files_count', 'deleted_data_files_count', 'partitions',
       'added_rows_count', 'existing_rows_count', 'deleted_rows_count'],
      dtype='object')

In [79]:
avro_tempdf['partitions']

0    [(False, False, [56, 3, 4, 0], [56, 3, 4, 0])]
Name: partitions, dtype: object

In [81]:
avro_tempdf['added_rows_count']

0    1
Name: added_rows_count, dtype: int64

In [82]:
avro_tempdf['existing_rows_count']

0    0
Name: existing_rows_count, dtype: int64

In [84]:
avro_tempdf['added_data_files_count']

0    1
Name: added_data_files_count, dtype: int32

In [85]:
print("Showing " + metadata_file_list[2])
json_tempdf = spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[2]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/customer_table/metadata/00002-30665850-e97b-46dd-a4f1-2dee67bbe972.metadata.json


In [86]:
json_tempdf.columns

Index(['current-schema-id', 'current-snapshot-id', 'default-sort-order-id',
       'default-spec-id', 'format-version', 'last-column-id',
       'last-partition-id', 'last-updated-ms', 'location', 'metadata-log',
       'partition-spec', 'partition-specs', 'properties', 'refs', 'schema',
       'schemas', 'snapshot-log', 'snapshots', 'sort-orders', 'table-uuid'],
      dtype='object')

In [90]:
json_tempdf['current-schema-id']

0    0
Name: current-schema-id, dtype: int64

In [92]:
list(json_tempdf['snapshots'])

[[Row(manifest-list='s3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/customer_table/metadata/snap-3986362509610264284-1-385f35b6-3a5a-4d30-a7a1-156fa6a8c7b2.avro', parent-snapshot-id=None, schema-id=0, snapshot-id=3986362509610264284, summary=Row(added-data-files='1', added-files-size='1106', added-records='1', changed-partition-count='1', deleted-data-files=None, deleted-records=None, operation='append', removed-files-size=None, spark.app.id='spark-application-1684536915149', total-data-files='1', total-delete-files='0', total-equality-deletes='0', total-files-size='1106', total-position-deletes='0', total-records='1'), timestamp-ms=1684537347083),
  Row(manifest-list='s3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/customer_table/metadata/snap-6274129147237535833-1-021df821-94c9-424f-af1a-51fa31667010.avro', parent-snapshot-id=3986362509610264284, schema-id=0, snapshot-id=6274129147237535833, summary=Row(added-data-files='4', added-files-size='4760', a

In [93]:
json_tempdf['partition-spec']

0    [(1000, dob_hour, 4, hour)]
Name: partition-spec, dtype: object

In [98]:
list(json_tempdf['partition-specs'])

[[Row(fields=[Row(field-id=1000, name='dob_hour', source-id=4, transform='hour')], spec-id=0)]]

In [ ]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.all_data_files;").show()

### Partition Evolution

Spark partitioning is a way to split the data into multiple partitions so that you can execute transformations on multiple partitions in parallel which allows completing the job faster. You can also write partitioned data into a file system (multiple sub-directories) for faster reads by downstream systems.

Spark has several partitioning methods to achieve parallelism, based on your need, you should choose which one to use.

Creating New Data to Test Partition Evolution

In [7]:
from pyspark.sql.types import LongType, IntegerType, StringType

import dbldatagen as dg

shuffle_partitions_requested = 20
device_population = 100000
data_rows = 20 * 1000000
#partitions_requested = 20

spark.conf.set("spark.sql.shuffle.partitions", shuffle_partitions_requested)

country_codes = [
    "CN", "US", "FR", "CA", "IN", "JM", "IE", "PK", "GB", "IL", "AU", 
    "SG", "ES", "GE", "MX", "ET", "SA", "LB", "NL", "IT"
]
#country_weights = [
#    1300, 365, 67, 38, 1300, 3, 7, 212, 67, 9, 25, 6, 47, 83, 
#    126, 109, 58, 8, 17,
#]

manufacturers = [
    "Delta corp", "Xyzzy Inc.", "Lakehouse Ltd", "Acme Corp", "Embanks Devices",
]

lines = ["delta", "xyzzy", "lakehouse", "gadget", "droid"]

testDataSpec = (
    dg.DataGenerator(spark, name="device_data_set", rows=data_rows) 
                     #,partitions=partitions_requested)
    .withIdOutput()
    # we'll use hash of the base field to generate the ids to
    # avoid a simple incrementing sequence
    .withColumn("internal_device_id", "long", minValue=0x1000000000000, 
                uniqueValues=device_population, omit=True, baseColumnType="hash",
    )
    # note for format strings, we must use "%lx" not "%x" as the
    # underlying value is a long
    .withColumn(
        "device_id", "string", format="0x%013x", baseColumn="internal_device_id"
    )
    # the device / user attributes will be the same for the same device id
    # so lets use the internal device id as the base column for these attribute
    .withColumn("country", "string", values=country_codes, #weights=country_weights, 
                baseColumn="internal_device_id")
    .withColumn("manufacturer", "string", values=manufacturers, 
                baseColumn="internal_device_id", )
    # use omit = True if you don't want a column to appear in the final output
    # but just want to use it as part of generation of another column
    .withColumn("line", "string", values=lines, baseColumn="manufacturer", 
                baseColumnType="hash", omit=True )
    .withColumn("model_ser", "integer", minValue=1, maxValue=11, baseColumn="device_id", 
                baseColumnType="hash", omit=True, )
    .withColumn("model_line", "string", expr="concat(line, '#', model_ser)", 
                baseColumn=["line", "model_ser"] )
    .withColumn("event_type", "string", 
                values=["activation", "deactivation", "plan change", "telecoms activity", 
                        "internet activity", "device error", ],
                random=True)
    .withColumn("event_ts", "timestamp", begin="2020-01-01 01:00:00", 
                end="2020-12-31 23:59:00", 
                interval="1 minute", random=True )
)

dfTestData = testDataSpec.build()

display(dfTestData)

INFO: Version : VersionInfo(major='0', minor='2', patch='1', release='', build='')


DataFrame[id: bigint, device_id: string, country: string, manufacturer: string, model_line: string, event_type: string, event_ts: timestamp]

In [8]:
dfTestData.head()

Row(id=0, device_id='0x100000001281d', country='US', manufacturer='Xyzzy Inc.', model_line='lakehouse#10', event_type='internet activity', event_ts=datetime.datetime(2020, 11, 4, 3, 44))

In [9]:
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")

In [10]:
spark.sql("DROP TABLE IF EXISTS spark_catalog.lakehouse.partition_evol_tbl PURGE")

DataFrame[]

In [11]:
#dfTestData.groupBy("country").count().show()

In [12]:
#dfTestData.rdd.getNumPartitions()

Iceberg requires the data to be sorted according to the partition spec per task (Spark partition) in prior to write against partitioned table. This applies both Writing with SQL and Writing with DataFrames.

In [13]:
dfTestData.sortWithinPartitions("country").writeTo("spark_catalog.lakehouse.p_evol_tbl").partitionedBy("country").using("iceberg").create()#.append()#replace()#overwritePartitions()#create()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.PARTITIONS").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.files").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.manifests").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.all_manifests").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.all_data_files").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.snapshots").show()

Adding a partition field is a metadata operation and does not change any of the existing table data. New data will be written with the new partitioning, but existing data will remain in the old partition layout. Old data files will have null values for the new partition fields in metadata tables.

In [14]:
print("TABLE PARTITIONS BEFORE ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.p_evol_tbl.PARTITIONS").show()

TABLE PARTITIONS BEFORE ALTER PARTITION STATEMENT: 
+---------+------------+----------+-------+
|partition|record_count|file_count|spec_id|
+---------+------------+----------+-------+
|     {CN}|     1000100|        10|      0|
|     {CA}|     1000301|        10|      0|
|     {IT}|      999401|        10|      0|
|     {IL}|     1000474|        10|      0|
|     {NL}|     1000498|        10|      0|
|     {AU}|     1000859|        10|      0|
|     {GB}|      999318|        10|      0|
|     {US}|     1000098|        10|      0|
|     {SG}|      999532|        10|      0|
|     {LB}|      998789|        10|      0|
|     {IN}|      999328|        10|      0|
|     {IE}|      998731|        10|      0|
|     {FR}|      999338|        10|      0|
|     {ES}|     1000299|        10|      0|
|     {GE}|      999322|        10|      0|
|     {MX}|     1001198|        10|      0|
|     {PK}|     1002144|        10|      0|
|     {JM}|     1000840|        10|      0|
|     {SA}|      998932|

In [15]:
print("ADD PARTITION BY EVENT TIMESTAMP MONTHS: ")
print("ALTER TABLE spark_catalog.lakehouse.p_evol_tbl ADD PARTITION FIELD months(event_ts)")
spark.sql("ALTER TABLE spark_catalog.lakehouse.p_evol_tbl ADD PARTITION FIELD months(event_ts)")
#spark.sql("ALTER TABLE spark_catalog.lakehouse.part_evol_tbl REPLACE PARTITION FIELD hours(dob) WITH state")
#spark.sql("ALTER TABLE prod.db.sample ADD PARTITION FIELD month")

#ALTER TABLE spark_catalog.lakehouse.part_evol_tbl ADD PARTITION FIELD days(event_ts)

ADD PARTITION BY BUCKETING DEVICE ID:
ALTER TABLE spark_catalog.lakehouse.p_evol_tbl ADD PARTITION FIELD months(event_ts)


DataFrame[]

In [16]:
print("TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.p_evol_tbl.PARTITIONS").show()

TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: 


+---------+------------+----------+-------+
|partition|record_count|file_count|spec_id|
+---------+------------+----------+-------+
|     {US}|     1000098|        10|      0|
|     {LB}|      998789|        10|      0|
|     {FR}|      999338|        10|      0|
|     {GE}|      999322|        10|      0|
|     {IN}|      999328|        10|      0|
|     {SG}|      999532|        10|      0|
|     {IE}|      998731|        10|      0|
|     {JM}|     1000840|        10|      0|
|     {CN}|     1000100|        10|      0|
|     {SA}|      998932|        10|      0|
|     {AU}|     1000859|        10|      0|
|     {IT}|      999401|        10|      0|
|     {ET}|     1000498|        10|      0|
|     {PK}|     1002144|        10|      0|
|     {GB}|      999318|        10|      0|
|     {MX}|     1001198|        10|      0|
|     {NL}|     1000498|        10|      0|
|     {CA}|     1000301|        10|      0|
|     {IL}|     1000474|        10|      0|
|     {ES}|     1000299|        

In [17]:
appendDf = dfTestData.sample(fraction=0.3, seed=3)

In [18]:
appendDf.dtypes

[('id', 'bigint'),
 ('device_id', 'string'),
 ('country', 'string'),
 ('manufacturer', 'string'),
 ('model_line', 'string'),
 ('event_type', 'string'),
 ('event_ts', 'timestamp')]

In [19]:
appendDf.rdd.getNumPartitions()

10

In [20]:
appendDf.show()

+---+---------------+-------+---------------+------------+-----------------+-------------------+
| id|      device_id|country|   manufacturer|  model_line|       event_type|           event_ts|
+---+---------------+-------+---------------+------------+-----------------+-------------------+
|  0|0x100000001281d|     US|     Xyzzy Inc.|lakehouse#10|internet activity|2020-11-04 03:44:00|
|  4|0x1000000003674|     SA|     Xyzzy Inc.| lakehouse#6|       activation|2020-07-05 01:45:00|
|  5|0x100000001492c|     IN|Embanks Devices| lakehouse#7|telecoms activity|2020-01-24 22:31:00|
| 17|0x1000000010aa2|     MX|Embanks Devices| lakehouse#1|     deactivation|2020-09-29 01:01:00|
| 20|0x1000000001f27|     SG|     Xyzzy Inc.|lakehouse#11|      plan change|2020-10-26 00:21:00|
| 23|0x10000000039b4|     GB|      Acme Corp|    droid#10|     deactivation|2020-11-11 02:00:00|
| 28|0x1000000003a10|     CN|     Delta corp|    gadget#1|       activation|2020-10-28 03:30:00|
| 29|0x1000000014cd0|     SA| 

In [21]:
appendDf.sortWithinPartitions("country").show()

+----+---------------+-------+------------+----------+-----------------+-------------------+
|  id|      device_id|country|manufacturer|model_line|       event_type|           event_ts|
+----+---------------+-------+------------+----------+-----------------+-------------------+
| 132|0x1000000008146|     AU|  Delta corp|  gadget#2|     device error|2020-04-24 10:00:00|
| 150|0x100000001540e|     AU|  Delta corp| gadget#11|internet activity|2020-10-11 06:21:00|
| 217|0x100000000582e|     AU|  Delta corp|  gadget#2|telecoms activity|2020-04-11 14:24:00|
| 279|0x10000000183b6|     AU|  Delta corp| gadget#10|      plan change|2020-03-31 04:31:00|
| 368|0x100000000c2d2|     AU|  Delta corp|  gadget#1|       activation|2020-03-25 11:11:00|
| 489|0x10000000185d2|     AU|  Delta corp|  gadget#8|      plan change|2020-12-11 13:21:00|
| 515|0x100000001729a|     AU|  Delta corp|  gadget#3|telecoms activity|2020-04-12 14:19:00|
| 804|0x100000000d1be|     AU|  Delta corp|  gadget#2|      plan chang

In [22]:
#appendDf.sortWithinPartitions("country", "month(event_ts)").show()

In [23]:
appendDf.sortWithinPartitions("country").writeTo("spark_catalog.lakehouse.p_evol_tbl").using("iceberg").append() #.append()#replace()#overwritePartitions()#create()

In [24]:
print("TABLE PARTITIONS AFTER APPEND: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.p_evol_tbl.PARTITIONS").show(100)

TABLE PARTITIONS AFTER APPEND: 
+----------+------------+----------+-------+
| partition|record_count|file_count|spec_id|
+----------+------------+----------+-------+
| {MX, 610}|       24539|        10|      1|
| {CA, 606}|       25261|        10|      1|
| {ES, 603}|       24582|        10|      1|
| {MX, 601}|       23816|        10|      1|
|{SG, null}|      999532|        10|      0|
| {FR, 600}|       25141|        10|      1|
| {IE, 600}|       25480|        10|      1|
| {IE, 603}|       24541|        10|      1|
| {SG, 600}|       25147|        10|      1|
| {SG, 604}|       25366|        10|      1|
| {IL, 601}|       23763|        10|      1|
| {MX, 611}|       25176|        10|      1|
| {IE, 605}|       24326|        10|      1|
| {LB, 607}|       25535|        10|      1|
| {IN, 611}|       25312|        10|      1|
| {JM, 601}|       24042|        10|      1|
|{IE, null}|      998731|        10|      0|
| {GE, 611}|       25223|        10|      1|
| {CA, 608}|       2466

Dropping a partition field is a metadata operation and does not change any of the existing table data. New data will be written with the new partitioning, but existing data will remain in the old partition layout.

In [74]:
spark.sql("ALTER TABLE spark_catalog.lakehouse.part_evol_tbl DROP PARTITION FIELD bucket(16, device_id)")

DataFrame[]

In [75]:
print("TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.PARTITIONS").show()

CAR SALES TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: 


+----------+------------+----------+-------+
| partition|record_count|file_count|spec_id|
+----------+------------+----------+-------+
|{CN, null}|    10000790|        10|      0|
|{IT, null}|    10002099|        10|      0|
|{US, null}|     9998801|        10|      0|
|{AU, null}|     9997695|        10|      0|
|{GE, null}|    10005212|        10|      0|
|{PK, null}|    10002237|        10|      0|
|{ES, null}|    10001299|        10|      0|
|{SA, null}|     9998077|        10|      0|
|{ET, null}|    10001822|        10|      0|
|{NL, null}|     9998304|        10|      0|
|{IN, null}|    10004181|        10|      0|
|{MX, null}|    10002210|        10|      0|
|{GB, null}|     9993405|        10|      0|
|{IE, null}|     9998066|        10|      0|
|{CA, null}|     9997659|        10|      0|
|{LB, null}|     9996707|        10|      0|
|{JM, null}|    10000921|        10|      0|
|{IL, null}|     9998131|        10|      0|
|{FR, null}|    10003914|        10|      0|
|{SG, null

##### Only json files have been added (one per each time you repartitioned) but Avro files have stayed the same

In [57]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

print("Current Metadata Files: \n")

for object_summary in my_bucket.objects.filter(Prefix=metadata_path+"/metadata"):
    #print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)
    
metadata_file_list

NameError: name 'boto3' is not defined

In [ ]:
spark.sql("CREATE TABLE IF NOT EXISTS customer_table (id BIGINT, state STRING, country STRING, dob TIMESTAMP) USING iceberg PARTITIONED BY ( hours(dob))")

In [110]:
spark.sql("SELECT HOUR(dob) FROM spark_catalog.lakehouse.customer_table").show()

+---------+
|hour(dob)|
+---------+
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
+---------+



In [107]:
spark.sql("SELECT DAY(dob) FROM spark_catalog.lakehouse.customer_table").show()

+--------+
|day(dob)|
+--------+
|       1|
|       1|
|       1|
|       1|
|       2|
|       3|
|       3|
|       4|
|       4|
+--------+



### Dropping Tables

In [ ]:
spark.sql("DROP TABLE IF EXISTS lakehouse.staging")

Validate that the metadata folder is now empty but the data folder still retains parquet files.

![alt text](../img/s3_droptable_1.png)

![alt text](../img/s3_droptable_2.png)

![alt text](../img/s3_droptable_3.png)

In [ ]:
spark.sql("ALTER TABLE lakehouse.customers_table\
            SET TBLPROPERTIES ('format-version' = '2')")

In [26]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

for object_summary in my_bucket.objects.filter(Prefix=metadata_path):
    print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00000-8f9c2e19-d640-404e-8fc8-c4287b667740.metadata.json



In [27]:
print("Showing " + metadata_file_list[3])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[3]).toPandas()

IndexError: list index out of range